In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [2]:
X = np.random.random((10000, 2))

In [3]:
X.shape

(10000, 2)

In [4]:
M = np.dot(X, [[2, 3, 1], 
               [4, 2, 1]]) + 1

In [5]:
M.shape

(10000, 3)

In [6]:
Y = np.dot(M, [[1], 
               [5], 
               [2]]) + 1

In [7]:
Y.shape

(10000, 1)

## base model

In [8]:
base_model = Sequential([
    Dense(3, input_shape=(2,))
])

In [9]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [10]:
base_model.compile(optimizer='sgd', loss='mse')
base_model.fit(X, M, epochs=10, verbose=0)

In [11]:
base_model.evaluate(X, M)

 9440/10000 [===========================>..] - ETA: 0s

0.013976614066958427

freeze base_model weights

In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
for layer in base_model.layers:
    print(layer.get_weights())

[array([[ 1.86094177,  2.65433908,  1.11577344],
       [ 3.44748139,  1.81652141,  1.02085984]], dtype=float32), array([ 1.36749578,  1.28072572,  0.92756146], dtype=float32)]


In [14]:
pred_mid = base_model.predict(X)

In [15]:
base_model.layers[0].get_weights()

[array([[ 1.86094177,  2.65433908,  1.11577344],
        [ 3.44748139,  1.81652141,  1.02085984]], dtype=float32),
 array([ 1.36749578,  1.28072572,  0.92756146], dtype=float32)]

## base_model output as dense layer input

In [16]:
lm = Sequential([
    Dense(1, input_shape=(3,))
])
lm.compile(optimizer='sgd', loss='mse')

In [17]:
lm.fit(pred_mid, Y, epochs=5, verbose=1)

Epoch 1/5
10000/10000 [==============================] - 0s - loss: 2.5169     
Epoch 2/5
10000/10000 [==============================] - 0s - loss: 0.2601     
Epoch 3/5
10000/10000 [==============================] - 0s - loss: 0.0888     
Epoch 4/5
10000/10000 [==============================] - 0s - loss: 0.0343     
Epoch 5/5
10000/10000 [==============================] - 0s - loss: 0.0143     


In [18]:
for layer in lm.layers:
    print(layer.get_weights())

[array([[ 1.35665238],
       [ 5.78818941],
       [ 0.73444736]], dtype=float32), array([-0.72404605], dtype=float32)]


In [19]:
pred1 = lm.predict(pred_mid)

In [20]:
lm.evaluate(pred_mid, Y)

 9280/10000 [==========================>...] - ETA: 0s

0.009925397582352162

## put in one model 

In [21]:
x = base_model.output
predictions = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='sgd', loss='mse')

In [22]:
model.fit(X, Y, epochs=5, verbose=1)

Epoch 1/5
10000/10000 [==============================] - 0s - loss: 2.7336     
Epoch 2/5
10000/10000 [==============================] - 0s - loss: 0.1724     
Epoch 3/5
10000/10000 [==============================] - 0s - loss: 0.0718     
Epoch 4/5
10000/10000 [==============================] - 0s - loss: 0.0311     
Epoch 5/5
10000/10000 [==============================] - 0s - loss: 0.0137     


In [23]:
pred2 = model.predict(X)

In [24]:
model.evaluate(X, Y)

 9472/10000 [===========================>..] - ETA: 0s

0.0088137792930006974

In [25]:
for layer in model.layers:
    print(layer.get_weights())

[]
[array([[ 1.86094177,  2.65433908,  1.11577344],
       [ 3.44748139,  1.81652141,  1.02085984]], dtype=float32), array([ 1.36749578,  1.28072572,  0.92756146], dtype=float32)]
[array([[ 1.18471217],
       [ 5.38311481],
       [ 1.98310089]], dtype=float32), array([-1.12827611], dtype=float32)]


In [26]:
base_model.layers[0].get_weights()

[array([[ 1.86094177,  2.65433908,  1.11577344],
        [ 3.44748139,  1.81652141,  1.02085984]], dtype=float32),
 array([ 1.36749578,  1.28072572,  0.92756146], dtype=float32)]

In [27]:
np.all(model.layers[1].get_weights()[0] == base_model.layers[0].get_weights()[0])

True

In [28]:
np.all(model.layers[1].get_weights()[1] == base_model.layers[0].get_weights()[1])

True

In [29]:
model.layers[2].get_weights()

[array([[ 1.18471217],
        [ 5.38311481],
        [ 1.98310089]], dtype=float32), array([-1.12827611], dtype=float32)]

In [30]:
lm.layers[0].get_weights()

[array([[ 1.35665238],
        [ 5.78818941],
        [ 0.73444736]], dtype=float32), array([-0.72404605], dtype=float32)]

### output of model at each layer

In [31]:
# https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

# Testing
layer_outs = [func([X, 1.]) for func in functors]

In [32]:
np.allclose(X, layer_outs[0])

True

In [33]:
np.all(pred_mid==layer_outs[1])

True

In [34]:
np.all(pred2==layer_outs[2])

True